In [1]:
import os,pickle
os.environ["CUDA_VISIBLE_DEVICES"]="2"
import numpy as np
import torch
import pandas
from tqdm import tqdm
from transformers import BitsAndBytesConfig,LlamaTokenizer,LlamaForCausalLM 
from Custom_bert_generate2 import CustomDataset, Model,get_generated_sentence_data, get_accuracy_by_bert
from tuning import get_model_architecture,get_model

device = 'cuda'

tuning_name = "lora"
model_name = "llama"

tokenizer,model = get_model(model_name,device)

model = get_model_architecture(tuning_name,model_name, model)

model.load_state_dict(torch.load(tuning_name+'.pt'))

model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): lora.Embedding(
          (base_layer): Embedding(32000, 4096)
          (lora_dropout): ModuleDict(
            (default): Identity()
          )
          (lora_A): ModuleDict()
          (lora_B): ModuleDict()
          (lora_embedding_A): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 64x32000 (cuda:0)])
          (lora_embedding_B): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 4096x64 (cuda:0)])
        )
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
        

In [2]:

with open('FT_input_datasets_llama.pickle', 'rb') as fr:
    data = pickle.load(fr)
data=data[:200]

In [3]:
data = get_generated_sentence_data(model, tokenizer, device, data)
get_accuracy_by_bert(data,device)

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [3:20:42<00:00, 60.21s/it]
/home/wooseok/peft/dailydialogue/llama/Custom_bert_generate2.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['act'] = data['act']-1
valid_step: 100%|███████████████████████████████| 10/10 [00:05<00:00,  1.81it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       device='cuda:0')
accuracy: 41.0


41.0

In [4]:
data

,context,reference,act,act_label,input_tokens,generated
0,"[Hey man , you wanna buy some weed ? ]",Some what ?,2,Question,"[1, 894, 2, 1, 18637, 767, 1919, 366, 281, 971...",everybody's talking about the 2012 olympics\nP...
1,"[Hey man , you wanna buy some weed ? , Some w...","Weed ! You know ? Pot , Ganja , Mary Jane som...",3,Directive,"[1, 8797, 573, 2, 1, 18637, 767, 1919, 366, 28...",obviously I'm not sure what you're talking abo...
2,"[Hey man , you wanna buy some weed ? , Some w...","Oh , umm , no thanks .",4,Commissive,"[1, 1876, 790, 573, 2, 1, 18637, 767, 1919, 36...","sierp 2013. 11:00 - 14:00.\nWielka Brytania, L..."
3,"[Hey man , you wanna buy some weed ? , Some w...",I also have blow if you prefer to do a few li...,3,Directive,"[1, 8797, 573, 2, 1, 18637, 767, 1919, 366, 28...","obviously, this is a bit of a tongue-in-cheek ..."
4,[The taxi drivers are on strike again . ],What for ?,2,Question,"[1, 894, 2, 1, 450, 8818, 29875, 18563, 526, 3...",nobody likes a quitter.\nI’m not a quitter.\nI...
...,...,...,...,...,...,...
195,"[There's a new girl in school , have you seen ...",I think that she is very pretty .,1,Inform,"[1, 15162, 2, 1, 1670, 29915, 29879, 263, 716,...",nobody is going to tell you what to do.\nand y...
196,"[There's a new girl in school , have you seen ...",Tell me how she looks .,1,Inform,"[1, 15162, 2, 1, 1670, 29915, 29879, 263, 716,...","февраль 2018 г.\nA few months ago, we publishe..."
197,"[There's a new girl in school , have you seen ...",She's kind of short .,1,Inform,"[1, 15162, 2, 1, 1670, 29915, 29879, 263, 716,...",nobody has been able to find a way to make a p...
198,[Take a seat inside and see what you think.So ...,"Yes , and I want to buy the insurance too . I...",1,Inform,"[1, 15162, 2, 1, 11190, 263, 12949, 2768, 322,...",Hinweis: Dieser Artikel ist nur auf Englisch v...


In [5]:
import pickle
with open('generated_reference_{}.pickle'.format(tuning_name),'wb') as fw:
    pickle.dump(data, fw)